# Tabular radiomics demo

Trains and evaluates three quantum(-inspired) models on the provided **real tabular radiomics** example.

Outputs are written into `Results/real_train/`.

This notebook runs **without SHAP/LIME** (those are intentionally separated).

In [ ]:
from pathlib import Path
import pandas as pd

REPO = Path().resolve()
assert (REPO/'qnm_qai.py').exists(), 'Run Jupyter from the repo root'

# Build real_train.csv / real_infer.csv from raw FDB/LDB (demo_data/tabular/raw)
!python examples/build_tabular_from_fdb_ldb.py

train_csv = Path('demo_data/tabular/real_train.csv')
infer_csv = Path('demo_data/tabular/real_infer.csv')
print('Train:', train_csv, 'exists=', train_csv.exists())
print('Infer:', infer_csv, 'exists=', infer_csv.exists())

df = pd.read_csv(train_csv)
print('Shape:', df.shape)
df.head()

## Train + evaluate (no explanations)

Runtime is dominated by:
- `pl_kernel_svm`: building kernel matrices (many simulator circuit evaluations)
- `pl_qcnn_*`: training epochs (many circuit evaluations per epoch)

The flags below keep this short.

In [ ]:
!python qnm_qai.py run \
  --input demo_data/tabular/real_train.csv \
  --infer demo_data/tabular/real_infer.csv \
  --input-type tabular \
  --methods pl_kernel_svm,pl_qcnn_alt,pl_qcnn_muw \
  --results-dir Results \
  --test-size 0.25 \
  --qcnn-epochs 2 \
  --qcnn-lr 0.05 \
  --max-samples-per-method 80 \
  --no-explain

## View summary metrics (per method)

In [ ]:
import pandas as pd

summary = Path('Results')/'real_train__results.csv'
df_sum = pd.read_csv(summary)
df_sum

## Inspect per-case test predictions

Each method writes its own `predictions/test.csv`.

In [ ]:
import pandas as pd
from pathlib import Path

base = Path('Results')/'real_train'
for method in ['pl_kernel_svm','pl_qcnn_alt','pl_qcnn_muw']:
    p = base/method/'predictions'/'test.csv'
    print('\n', method, p)
    dfp = pd.read_csv(p)
    display(dfp.head(10))